# Automatizovaná detekcia veku a pohlavia z fotky s využitím CNN.

_**Authors:** (Edvin Mako, Viliam Schrojf)_

## Motivation

Cieľom projektu je navrhnúť a realizovať konvolučńú neuronovu sieť s pomocou technológie TensorFlow, ktorá dokáže z fotky tváre osoby detegovať jej vybrané črty. Primárne sa sústredíme na detekciu veku a pohlavia. Pre tento účel sme našli aj vhodnú dátovú sadu, nad ktorou plánujeme model natrénovať. V rámci projektu, sa budeme snažiť experimentovať s rôznymi črtami, ktoré nám v predikcií veku pomôžu.

Myslíme si že táto úloha je dôležitá a môže mať uplatnenie v rôznych systémoch, kde je nutné zadať vek alebo dátum narodenia a fotku tváre. Správcovia systémov by týmto spôsobom overili správnosť zadaných údajov jednoducho.

V budúcnosti vieme náš model rozšíriť aj detekciu ďalších čŕt, ako napríklad: farba a typ vlasov, farba a umiestnenie očí a iných častí tváre (Nie je súčasťou tohto projektu).

Pri analýze tejto úlohy, sme sa zamysleli nad nasledovnými otázkami. Dokážeme navrhnúť podobnú konvolučnú sieť s menším počtom vrstiev, alebo iným usporiadanim ako VGG? Aká výkonná bude naša sieť a kolko ju budeme trénovať? Je VGG najlepším riešením podobných úloh?

## Related Work

Vychádzame z práce výskumného tímu pre počítačové videnia z ETH Zurich, ktorý podobné typy úloh riešia pomocou modifikácie VGG-16 siete predtrénovanej na datasete ImageNet. Ich prístup k riešení tejto úlohy bol najprv z fotky detekovať a vyrezať tvár, ktorá potom prešla sadou konvolučných neurónových sietí, ktoré estimovali vek osoby. Natrénovaná neurónová sieť bola optimizovaná na detekciu črt z daných fotiek. 

Tento tím nazbieral dáta celebrít z IMDB a wikipedie spolu z informáciami o ich veku, a ich zverejnil. Pre účel tohto projektu, budeme aj my tieto dáta používať [1].

Existujú aj podobné riešenia, ktoré modifikujú konečné vrstvy konvolučnej siete, ktorá bola predtrénovaná na detekciu tváre v obrázku na VGG-Face Modely. Zamenia iba niekolko z posledných vrstiev a tie trénujú. Prvé vrstvy ostatnú bez zmeny, ako boli natrénované inou skupinou. Aplikujú transfer learning [2].

## Datasets

K dispozícii máme veľký dataset s názvom “The IMDB-WIKI dataset” [1][3], ktorý obsahuje vyše 520.000 obrázkov rôznych celebrít. Každý obrázok obsahuje metainformácie s dátumom narodenia, menom osoby, rok kedy bola fotka vytvorená, oblasť tváre, pohlavie a dátum narodenia danej osoby, údaj ktorý využijeme na trénovanie neurónovej sieti.

Každý metaúdaj obsahuje tiež aj skóre na koľko je informácia o tvári a pohlavia spoľahlivá. V rámci prípravy datasetu vyberieme iba prvých 100 000 obrázkov, ktoré budú zoradené podľa najlepšieho skóre oblasti tváre. Následne tento dataset rozšíriť o natočené a naškálované obrázky s pridaným okrajom (paddingom).

## High-Level Solution Proposal

Ako prvú myšlienku o krokoch implementácie našej neurónovej siete, nájdeme v nasledujúcich riadkoch

1. Vyberieme iba nejaký počet najspoľahlivejších obrázkov.
2. Overíme pomocu jednoduchej analýzi, vytvoríme histogram veku a pohlavia. Najmladšiu a najstaršiu fotku. Určiť hranice modelu. Fotky detí od 0-16, alebo ľudí starších ako 90 rokov tam asi nebudú.
3. Rozšíríme dataset o zväčšené a zmenšené obrázky (scaling). Každý obrázok aj mierne natočíme.
4. Z vybraných fotiek najprv orežeme iba oblasť tváre.
5. Rozdelíme na testovaciu a trenovaciu vzorku.
6. Budeme experimentovať a porovnávať rôzne konfigurácie konvolučných vrstiev.
7. Na konci aplikujeme SOFTMAX funkciu. Použijeme podobnú konfiguraciu so 101 neuronmi pre vek {0, 1, 2, …, 99, 100} ako [3].
8. Niekolko predtrenovanych konfigurácii uložíme spolu s ich váhami a následne porovnáme a vyberieme najlepšiu konfiguráciu vrstiev.

### Ako možné experimenty a rozšírenia:

- Amotovať ďalšie časti tváre na obrázku
- Skúsiť pridať ďalšie črty, ako dĺžku vlasov, upravené obočie, peri, atď.
- Aplikovať transfer learning na nové črty
- Skúsiť pridať tváre ľudí, ktorý vyzerajú mladši alebo starší ako ich aktuálny vek a pozrieť sa na výsledok predikcie.


Install addition packages: `pip install numpy scipy matplotlib ipython jupyter pandas sympy nose`

In [6]:
import scipy.io
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt

import tensorflow as tf
import tensorflow.keras as keras
import numpy as np

In [7]:
scipy.io.whosmat('dataset/wiki_crop/wiki.mat')

[('wiki', (1, 1), 'struct')]

In [8]:
wiki_mat = scipy.io.loadmat('dataset/wiki_crop/wiki.mat')

In [9]:
wiki_mat

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sat Jan 16 16:25:20 2016',
 '__version__': '1.0',
 '__globals__': [],
 'wiki': array([[(array([[723671, 703186, 711677, ..., 720620, 723893, 713846]], dtype=int32), array([[2009, 1964, 2008, ..., 2013, 2011, 2008]], dtype=uint16), array([[array(['17/10000217_1981-05-05_2009.jpg'], dtype='<U31'),
         array(['48/10000548_1925-04-04_1964.jpg'], dtype='<U31'),
         array(['12/100012_1948-07-03_2008.jpg'], dtype='<U29'), ...,
         array(['09/9998109_1972-12-27_2013.jpg'], dtype='<U30'),
         array(['00/9999400_1981-12-13_2011.jpg'], dtype='<U30'),
         array(['80/999980_1954-06-11_2008.jpg'], dtype='<U29')]],
       dtype=object), array([[1., 1., 1., ..., 1., 1., 0.]]), array([[array(['Sami Jauhojärvi'], dtype='<U15'),
         array(['Dettmar Cramer'], dtype='<U14'),
         array(['Marc Okrand'], dtype='<U11'), ...,
         array(['Michael Wiesinger'], dtype='<U17'),
         array(['Johann Grugger

## Note

Folder tree structure
```text
.
├── dataset
│   ├── imdb_crop
│   │   ├── 00
│   │   ├── 01
...
│   │   ├── 99
│   │   └── imdb.mat
│   ├── imdb_meta
│   │   └── imdb
│   ├── wiki
│   │   ├── 00
│   │   ├── 01
...
│   │   ├── 99
│   │   └── wiki.mat
│   └── wiki_crop
│       ├── 00
│       ├── 01
...
│       ├── 98
│       ├── 99
│       └── wiki.mat
├── Project.ipynb
└── README.md

306 directories, 6 files

```

## Reference

[1] Rasmus Rothe, Radu Timofte, and Luc Van Gool 2015. DEX: Deep EXpectation of apparent age from a single image. In IEEE International Conference on Computer Vision Workshops (ICCVW).

[2] Zakariya Qawaqneh and Arafat Abu Mallouh and Buket D. Barkana 2017. Deep Convolutional Neural Network for Age Estimation based on VGG-Face Model. CoRR, abs/1709.01664.

[3] Rasmus Rothe, Radu Timofte, and Luc Van Gool 2016. Deep expectation of real and apparent age from a single image without facial landmarks. International Journal of Computer Vision (IJCV).